# Guía práctica de PostgreSQL con Python: Implementando DDL, DML, DCL y TCL

En esta guía, utilizaremos **Python** para interactuar con una base de datos **PostgreSQL** y ejemplificar los comandos **DDL**, **DML**, **DCL** y **TCL** que mencionamos anteriormente. Usaremos el módulo `psycopg2` para conectarnos a PostgreSQL desde Python.

In [1]:
# Instalar psycopg2 en caso de no tenerlo
#!pip install psycopg2

# Contenido:
1. Conexión a PostgreSQL desde Python
2. DDL (Data Definition Language)
3. DML (Data Manipulation Language)
4. DCL (Data Control Language)
5. TCL (Transaction Control Language)
6. Visualización de la base de datos a través de un puerto
7. Cierre de conexión

## 1. Conexión a PostgreSQL desde Python

In [12]:
import psycopg2
from psycopg2 import sql

# Datos de conexión
conexion = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="Linda",
    port=5432, # Puerto por defecto de PostgreSQL,

)

# Crear un cursor para ejecutar comandos SQL
cursor = conexion.cursor()

## 2. DDL (Data Definition Language)

Los comandos DDL se utilizan para definir o modificar la estructura de la base de datos.

In [4]:
# Crear la tabla clientes
cursor.execute("""
    CREATE TABLE IF NOT EXISTS clientes (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(100) NOT NULL,
        email VARCHAR(100) UNIQUE NOT NULL,
        telefono VARCHAR(20),
        direccion VARCHAR(255)
    );
""")
# Confirmamos los cambios
conexion.commit()

## 2.1 Crear una tabla

Vamos a crear una tabla llamada `clientes`.

In [5]:
# Crear la tabla clientes
cursor.execute("""
    CREATE TABLE IF NOT EXISTS clientes (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(100) NOT NULL,
        email VARCHAR(100) UNIQUE NOT NULL,
        telefono VARCHAR(20),
        direccion VARCHAR(255)
    );
""")
# Confirmamos los cambios
conexion.commit()


## 2.2 Modificar una tabla
Agreguemos una columna fecha_registro a la tabla clientes.

In [6]:
# Agregar columna fecha_registro
cursor.execute("""
    ALTER TABLE clientes ADD COLUMN fecha_registro DATE DEFAULT CURRENT_DATE;
""")
conexion.commit()


## 2.3 Eliminar una tabla
Supongamos que queremos eliminar una tabla llamada antigua_tabla.

In [2]:
# Eliminar tabla antigua_tabla si existe
cursor.execute("""
    DROP TABLE IF EXISTS antigua_tabla;
""")
conexion.commit()


# 3. DML (Data Manipulation Language)
Los comandos DML permiten manipular los datos en las tablas.

## 3.1 Insertar datos
Insertaremos algunos registros en la tabla clientes.

In [13]:
# Insertar registros
cursor.execute("""
    INSERT INTO clientes (nombre, email, telefono, direccion)
    VALUES 
        ('Daniel Godoy', 'daniel1@example.com', '123456789', 'Calle Falsa 123'),
        ('María Pérez', 'maria2@example.com', '987654321', 'Avenida Siempreviva 742');
""")
conexion.commit()


## 3.2 Consultar datos
Seleccionaremos todos los clientes.

In [10]:
# Seleccionar todos los clientes
cursor.execute("""
    SELECT * FROM clientes;
""")
clientes = cursor.fetchall()
for cliente in clientes:
    print(cliente)


(1, 'Daniel Godoy', 'daniel@example.com', '123456789', 'Calle Falsa 123', datetime.date(2024, 9, 14))
(2, 'María Pérez', 'maria@example.com', '987654321', 'Avenida Siempreviva 742', datetime.date(2024, 9, 14))


## 3.3 Actualizar datos
Actualizaremos el email de un cliente.

In [ ]:
# Actualizar email de Daniel Godoy
cursor.execute("""
    UPDATE clientes
    SET email = 'daniel.godoy@example.com'
    WHERE nombre = 'Daniel Godoy';
""")
conexion.commit()


## 3.4 Eliminar datos
Eliminaremos a un cliente de la tabla.

In [12]:
# Eliminar a María Pérez
cursor.execute("""
    DELETE FROM clientes
    WHERE nombre = 'María Pérez';
""")
conexion.commit()


# 4. DCL (Data Control Language)
Los comandos DCL gestionan los permisos y el control de acceso a la base de datos.

## 4.1 Crear un nuevo usuario
Crearemos un nuevo usuario llamado `usuario_demo`.

In [ ]:
# Crear un nuevo usuario
cursor.execute("""
    CREATE USER usuario_demo WITH PASSWORD 'contraseña_demo';
""")
conexion.commit()


## 4.2 Otorgar permisos
Otorgaremos permisos de selección en la tabla clientes al usuario usuario_demo.

In [14]:
# Otorgar permisos
cursor.execute("""
    GRANT SELECT ON clientes TO usuario_demo;
""")
conexion.commit()


## 4.3 Revocar permisos
Revocaremos los permisos otorgados.

In [15]:
# Revocar permisos
cursor.execute("""
    REVOKE SELECT ON clientes FROM usuario_demo;
""")
conexion.commit()


# 5. TCL (Transaction Control Language)
Los comandos TCL controlan las transacciones, que son un conjunto de operaciones SQL que se ejecutan como una unidad única.

## 5.1 Usando COMMIT
Demostraremos cómo confirmar una transacción.

In [16]:
try:
    # Iniciar una transacción
    cursor.execute("BEGIN;")
    
    # Ejecutar varias operaciones
    cursor.execute("""
        INSERT INTO clientes (nombre, email, telefono, direccion)
        VALUES ('Carlos López', 'carlos@example.com', '555555555', 'Calle Luna 456');
    """)
    cursor.execute("""
        UPDATE clientes
        SET telefono = '111111111'
        WHERE nombre = 'Daniel Godoy';
    """)
    
    # Confirmar la transacción
    conexion.commit()
    print("Transacción completada exitosamente.")
except Exception as e:
    # Si ocurre un error, deshacer la transacción
    conexion.rollback()
    print("Error en la transacción:", e)


Transacción completada exitosamente.


## 5.2 Usando ROLLBACK
Provocaremos un error para ver cómo funciona `ROLLBACK`.

In [17]:
try:
    cursor.execute("BEGIN;")
    
    # Esta inserción fallará porque el email es UNIQUE y ya existe
    cursor.execute("""
        INSERT INTO clientes (nombre, email)
        VALUES ('Daniel Godoy', 'daniel.godoy@example.com');
    """)
    
    conexion.commit()
    print("Transacción completada exitosamente.")
except Exception as e:
    conexion.rollback()
    print("Transacción revertida debido a un error:", e)


Transacción revertida debido a un error: llave duplicada viola restricción de unicidad «clientes_email_key»
DETAIL:  Ya existe la llave (email)=(daniel.godoy@example.com).



# 6. Visualización de la base de datos a través de un puerto
Puedes utilizar herramientas como pgAdmin o Adminer para visualizar y gestionar tu base de datos a través de una interfaz web.

## 6.1 Usando pgAdmin
1. Instalar pgAdmin: Descarga e instala pgAdmin desde pgadmin.org.

2. Configurar conexión:
• Abre pgAdmin y crea una nueva conexión al servidor PostgreSQL.
• Utiliza `localhost` y el puerto `5432` (o el que estés utilizando).
•Ingresa tus credenciales de usuario y contraseña.
3. Explorar la base de datos:
•Navega por las tablas, vistas y otros objetos de tu base de datos.
•Puedes ejecutar consultas SQL desde la interfaz de pgAdmin.

# 7. Cierre de conexión
No olvides cerrar la conexión y el cursor al finalizar.

In [14]:
# Cerrar cursor y conexión
cursor.close()
conexion.close()

# AUTOR

**Andrés Daniel Godoy Ortiz**

Docente Universidad Externado de Colombia y Universidad de la Sabana